In [1]:
from selenium.common.exceptions import NoSuchElementException
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import re
from selenium.webdriver.common.keys import Keys
import time

In [3]:
os.system(r'start chrome --remote-debugging-port=9527 --user-data-dir="c:\Users\ElvisBY_Chen\OneDrive - ASUS\Documents\Python Scripts\news"')
options = Options()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9527")
google = webdriver.Chrome(options=options)

# search page


In [53]:
google.implicitly_wait(3)

In [4]:
google.get('https://www.bestbuy.com/#?intl=nosplash')
time.sleep(10)
google.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/a[2]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div/div/div[1]/div[2]/a[2]"}
  (Session info: chrome=118.0.5993.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66C878EF2+54786]
	(No symbol) [0x00007FF66C7E5612]
	(No symbol) [0x00007FF66C69A64B]
	(No symbol) [0x00007FF66C6DB79C]
	(No symbol) [0x00007FF66C6DB91C]
	(No symbol) [0x00007FF66C716D87]
	(No symbol) [0x00007FF66C6FBEAF]
	(No symbol) [0x00007FF66C714D02]
	(No symbol) [0x00007FF66C6FBC43]
	(No symbol) [0x00007FF66C6D0941]
	(No symbol) [0x00007FF66C6D1B84]
	GetHandleVerifier [0x00007FF66CBC7F52+3524194]
	GetHandleVerifier [0x00007FF66CC1D800+3874576]
	GetHandleVerifier [0x00007FF66CC15D7F+3843215]
	GetHandleVerifier [0x00007FF66C915086+694166]
	(No symbol) [0x00007FF66C7F0A88]
	(No symbol) [0x00007FF66C7ECA94]
	

In [88]:
google.find_element(By.XPATH, '//*[@id="gh-search-input"]').click()
google.find_element(By.XPATH, '//*[@id="gh-search-input"]').send_keys('ASUS')
google.find_element(By.XPATH, '//*[@id="gh-search-input"]').send_keys(Keys.ENTER)

In [91]:
html = google.find_element(By.TAG_NAME, 'html')
html.send_keys(Keys.END)
time.sleep(5)
html.send_keys(Keys.HOME)
time.sleep(3)
sku_items = google.find_elements(By.XPATH, '//*[@id="main-results"]/ol/li/div/div/div[not(contains(@class, "shop-display-ad"))]')

In [98]:
google.find_elements(By.XPATH, '//li[@class="page-item"]/a')[-1].get_attribute('href')

'https://www.bestbuy.com/site/searchpage.jsp?cp=19&id=pcat17071&st=asus'

In [86]:
# 创建一个空的DataFrame，用于存储结果
result_df = pd.DataFrame(columns=["Titles", "Product_URL", "Models", "Sku", "Star", "Reviews_num", "RR_URL", "Prices"])

rr_pattern = r'Rating (\d+\.\d+) out of 5 stars with (\d+) reviews'
cnt = -1

# 循环处理每个sku_items中的元素
for sku in sku_items:
    cnt += 1
    print(cnt)

    # 向下滚动页面
    html = google.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.PAGE_DOWN)

    try:
        # 获取商品标题
        title = sku.find_element(By.XPATH, ".//h4[@class='sku-title']").get_attribute('textContent')

        # 获取产品链接
        prod_url = sku.find_element(By.XPATH, ".//h4[@class='sku-title']/a").get_attribute('href')

        # 获取型号和SKU
        model = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[0].text
        SKUdigit = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[1].text

        # 获取星级和评论数
        rr = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']//p[@class='visually-hidden']").text
        match = re.search(rr_pattern, rr)
        if match:
            star = float(match.group(1))
            reviews_num = int(match.group(2))
        else:
            star = None
            reviews_num = None

        # 获取评论链接
        rr_url = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']/a").get_attribute('href')

        # 获取价格
        price = sku.find_element(By.XPATH, ".//div[@class='sku-list-item-price']//span[@aria-hidden='true']").text

        # 创建一行数据
        row = [title, prod_url, model, SKUdigit, star, reviews_num, rr_url, price]

        # 将一行数据添加到DataFrame中
        result_df.loc[cnt] = row

    except NoSuchElementException:
        # 如果出现NoSuchElementException，说明某些元素未找到，可以选择跳过或采取其他操作
        pass




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


C:\Users\ElvisBY_Chen\AppData\Local\Temp\ipykernel_12416\1066423935.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df.loc[cnt] = row


17
18
19
20
                                               Titles  \
0   ASUS - 15.6" OLED Laptop - Intel Core i9-13900...   
1   ASUS - Vivobook 16" Laptop - AMD Ryzen 7 5800H...   
2   ASUS - ROG Zephyrus G16 16" 165Hz Gaming Lapto...   
3   ASUS - TUF 15.6" Gaming Laptop - Intel Core i7...   
4   ASUS - 15.6" OLED Laptop - Intel Core i9-13900...   
5   ASUS - Zenbook 14X 14.5" 2.8K OLED Touch Lapto...   
6   ASUS - TUF Gaming A16 16" 165Hz Gaming Laptop ...   
7   ASUS - Vivobook 16 M1605 16" Laptop - AMD Ryze...   
8   ASUS - TUF Gaming A16 16" 165Hz Gaming Laptop ...   
9   ASUS - ROG Zephyrus M16 16" 240Hz Gaming Lapto...   
10  ASUS - Zenbook 14X 14.5" 2.8K OLED Touch Lapto...   
11  ASUS - ROG Zephyrus G14 14” 165Hz Gaming Lapto...   
12  ASUS - Zenbook 14X 14.5" 2.8K OLED Touch Lapto...   
13  ASUS - 15.6" OLED Laptop - Intel Core i7-13620...   
14  ASUS - ROG Zephyrus 16" FHD 165Hz Gaming Lapto...   
15  ASUS - ROG Flow X13 13.4” Touchscreen Gaming L...   
16  ASUS - TUF 15.6

In [87]:
result_df

,Titles,Product_URL,Models,Sku,Star,Reviews_num,RR_URL,Prices
0,"ASUS - 15.6"" OLED Laptop - Intel Core i9-13900...",https://www.bestbuy.com/site/asus-15-6-oled-la...,Q540VJ-I93050,6534578,4.6,324,https://www.bestbuy.com/site/asus-15-6-oled-la...,"$1,299.99"
1,"ASUS - Vivobook 16"" Laptop - AMD Ryzen 7 5800H...",https://www.bestbuy.com/site/asus-vivobook-16-...,M1603QA-R712512,6535990,4.7,388,https://www.bestbuy.com/site/asus-vivobook-16-...,$479.99
2,"ASUS - ROG Zephyrus G16 16"" 165Hz Gaming Lapto...",https://www.bestbuy.com/site/asus-rog-zephyrus...,GU603VV-G16.I74060,6535496,4.6,136,https://www.bestbuy.com/site/asus-rog-zephyrus...,"$1,449.99"
3,"ASUS - TUF 15.6"" Gaming Laptop - Intel Core i7...",https://www.bestbuy.com/site/asus-tuf-15-6-gam...,FX507ZI-F15.I74070,6535503,4.7,142,https://www.bestbuy.com/site/asus-tuf-15-6-gam...,"$1,099.99"
4,"ASUS - 15.6"" OLED Laptop - Intel Core i9-13900...",https://www.bestbuy.com/site/asus-15-6-oled-la...,Q540VJ-I93050,6534578,4.6,324,https://www.bestbuy.com/site/asus-15-6-oled-la...,"$1,299.99"
5,"ASUS - Zenbook 14X 14.5"" 2.8K OLED Touch Lapto...",https://www.bestbuy.com/site/asus-zenbook-14x-...,Q410VA-EVO.I5512,6543527,4.7,207,https://www.bestbuy.com/site/asus-zenbook-14x-...,$799.99
6,"ASUS - TUF Gaming A16 16"" 165Hz Gaming Laptop ...",https://www.bestbuy.com/site/asus-tuf-gaming-a...,FA617NS-A16.R77600,6535499,4.6,302,https://www.bestbuy.com/site/asus-tuf-gaming-a...,"$1,099.99"
7,"ASUS - Vivobook 16 M1605 16"" Laptop - AMD Ryze...",https://www.bestbuy.com/site/asus-vivobook-16-...,M1605YA-ES74,6542092,4.8,5,https://www.bestbuy.com/site/asus-vivobook-16-...,$699.99
8,"ASUS - TUF Gaming A16 16"" 165Hz Gaming Laptop ...",https://www.bestbuy.com/site/asus-tuf-gaming-a...,FA617NS-A16.R77600,6535499,4.6,302,https://www.bestbuy.com/site/asus-tuf-gaming-a...,"$1,099.99"
9,"ASUS - ROG Zephyrus M16 16"" 240Hz Gaming Lapto...",https://www.bestbuy.com/site/asus-rog-zephyrus...,GU604VI-M16.I94070,6535501,4.6,384,https://www.bestbuy.com/site/asus-rog-zephyrus...,"$1,949.99"


# brand

In [ ]:
dellurl = 'https://www.bestbuy.com/site/brands/dell/pcmcat140500050010.c?id=pcmcat140500050010'
google.get(dellurl)

In [99]:
google.find_element(By.XPATH, '//*[@id="widget-d219661a-faa3-4ed3-adbf-9ba1853e1e88"]/div/div[2]/div/div[1]/div/a').click()

In [151]:
html = google.find_element(By.TAG_NAME, 'html')
html.send_keys(Keys.END)
time.sleep(5)
html.send_keys(Keys.HOME)
time.sleep(3)
brand_res_list = google.find_elements(By.XPATH, '//*[@id="main-results"]/ol/li/div/div/div[not(contains(@class, "shop-display-ad"))]')

In [155]:
brand_res_list[1].find_element(By.XPATH, ".//div[@class='sli-add-to-cart']//button").get_attribute('textContent')

'Add to Cart'

In [157]:
# 创建一个空的DataFrame，用于存储结果
brand_result_df = pd.DataFrame(columns=["Titles", "Product_URL", "Models", "Sku", "Star", "Reviews_num", "RR_URL", "Prices", "Save", "Previous Prices", 'AddtoCart button'])

rr_pattern = r'Rating (\d+\.\d+) out of 5 stars with (\d+) reviews'
cnt = -1

# 循环处理每个sku_items中的元素
for sku in brand_res_list:
    cnt += 1
    print(cnt)

    # 向下滚动页面
    html = google.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.PAGE_DOWN)

    
    
    try:
        # 获取商品标题
        title = sku.find_element(By.XPATH, ".//h4[@class='sku-title']").get_attribute('textContent')
        # 获取产品链接
        prod_url = sku.find_element(By.XPATH, ".//h4[@class='sku-title']/a").get_attribute('href')
        # 获取型号和SKU
        model = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[0].text
        SKUdigit = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[1].text
        # 获取评论链接
        rr_url = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']/a").get_attribute('href')
        # 获取价格
        price = sku.find_element(By.XPATH, ".//div[@class='sku-list-item-price']//span[@aria-hidden='true']").text
    except NoSuchElementException:
        continue

    # 获取星级和评论数
    rr = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']//p[@class='visually-hidden']").text
    match = re.search(rr_pattern, rr)
    if match:
        star = float(match.group(1))
        reviews_num = int(match.group(2))
    else:
        star = None
        reviews_num = None
        
    # save and old price
    try:
        save = sku.find_element(By.XPATH, ".//div[@class='pricing-price__savings pricing-price__savings--promo-red']").text
        old_price = sku.find_element(By.XPATH, './/div[@class="pricing-price__regular-price-content"]/div[1]').text
    except NoSuchElementException:
        save = 0
        old_price = price

    # addtocart status
    try:
        addtocart = sku.find_element(By.XPATH, ".//div[@class='sli-add-to-cart']//button").get_attribute('textContent')
    except NoSuchElementException:
        addtocart = 'Nothing'

    # 创建一行数据
    row = [title, prod_url, model, SKUdigit, star, reviews_num, rr_url, price, save, old_price, addtocart]

    # 将一行数据添加到DataFrame中
    brand_result_df.loc[cnt] = row




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [158]:
brand_result_df

,Titles,Product_URL,Models,Sku,Star,Reviews_num,RR_URL,Prices,Save,Previous Prices,AddtoCart button
2,"Dell - XPS 15 15.6"" 3.5K OLED Touch-Screen Lap...",https://www.bestbuy.com/site/dell-xps-15-15-6-...,XPS9530-9565SLV-PUS,6540609,4.8,45,https://www.bestbuy.com/site/dell-xps-15-15-6-...,"$2,799.99",0,"$2,799.99",Add to Cart
3,"Dell - G15 15.6"" Gaming Laptop - AMD Ryzen 5 ...",https://www.bestbuy.com/site/dell-g15-15-6-gam...,i5535-A933GRY-PUS,6552998,4.8,16,https://www.bestbuy.com/site/dell-g15-15-6-gam...,$799.99,Save $200,Was $999.99,Add to Cart
4,"Dell - Inspiron 16.0"" 2-in-1 Touch Laptop -13t...",https://www.bestbuy.com/site/dell-inspiron-16-...,i7630-7312SLV-PUS,6539902,4.7,303,https://www.bestbuy.com/site/dell-inspiron-16-...,"$1,099.99",0,"$1,099.99",Add to Cart
7,"Dell XPS 13 13.4"" FHD+ Laptop - 12th Gen Intel...",https://www.bestbuy.com/site/dell-xps-13-13-4-...,XPS9315-7725SKY-PUS,6540612,4.4,135,https://www.bestbuy.com/site/dell-xps-13-13-4-...,"$1,049.99",Save $300,"Was $1,349.99",Add to Cart
8,"Dell - Inspiron 14.0"" 2-in-1 Touch Laptop - 13...",https://www.bestbuy.com/site/dell-inspiron-14-...,i7430-5800SLV-PUS,6539904,4.6,126,https://www.bestbuy.com/site/dell-inspiron-14-...,$649.99,Save $80,Was $729.99,Add to Cart
9,"Dell - XPS 15 15.6"" FHD+ Laptop - Intel Evo i7...",https://www.bestbuy.com/site/dell-xps-15-15-6-...,XPS9530-7718SLV-PUS,6540611,4.6,93,https://www.bestbuy.com/site/dell-xps-15-15-6-...,"$1,499.99",0,"$1,499.99",Add to Cart
10,"Dell G15 15.6"" FHD 120Hz Gaming Laptop - Intel...",https://www.bestbuy.com/site/dell-g15-15-6-fhd...,G5530-7527BLK-PUS,6541333,4.4,59,https://www.bestbuy.com/site/dell-g15-15-6-fhd...,$849.99,Save $300,"Was $1,149.99",Add to Cart
11,"Dell - Inspiron 16.0"" 2-in-1 OLED Touch Laptop...",https://www.bestbuy.com/site/dell-inspiron-16-...,i7630-7305BLU-PUS,6539903,4.7,63,https://www.bestbuy.com/site/dell-inspiron-16-...,"$1,499.99",0,"$1,499.99",Add to Cart
14,"Dell - Inspiron 14.0"" 2-in-1 Touch Laptop - 13...",https://www.bestbuy.com/site/dell-inspiron-14-...,i7430-7374SLV-PUS,6539910,4.6,500,https://www.bestbuy.com/site/dell-inspiron-14-...,$999.99,0,$999.99,Add to Cart
16,Dell - Inspiron 2-in-1 14” FHD+ Touch Laptop –...,https://www.bestbuy.com/site/dell-inspiron-2-i...,i7425-A242PBL-PUS,6503229,4.4,631,https://www.bestbuy.com/site/dell-inspiron-2-i...,$559.99,Save $240,Reg $799.99,Unavailable Nearby
